# Составим список всех возможных значений фильтра

In [131]:
list_d_otverstiy = [98,100,105,
                    108,110,112,
                   114,114.3,115,118,120,127,130,139,139.7,150,160,170,180,205]

In [132]:
list_otverstiy = [x for x in range(3,7)]

In [133]:
list_shir_oboda = [0.5*x for x in range(9,23)]

In [134]:
list_diametr = [x for x in range(13,23)]

# Создадим лист с различными вариантами написания PCD

In [135]:
list_pcd = []
for i in list_otverstiy:
    for j in list_d_otverstiy:
        a = str(i)+'x'+str(j)
        b = str(i)+'х'+str(j)
        c = str(i)+' '+str(j)
        list_pcd.append(a)
        list_pcd.append(b)
        list_pcd.append(c)

# Создадим лист со всеми возможными комбинациями параметров
Этот лист нужен нам для генерации ключевых фраз

In [136]:
%time
list_pcd4 = []
a = [1,2,3,4,5,6,7]
for i in list_shir_oboda:
    for k in list_diametr:
        for j in list_pcd:
            a[0] = str(j)+';;'
            a[1] = str(j)+';'+str(i)+';'
            a[2] = str(j)+';;'+str(k)
            a[3] = str(j)+';'+str(i)+';'+str(k)
            a[4] = ';'+str(i)+';'+str(k)
            a[5] = ';'+str(i)+';'
            a[6] = ';;'+str(k)
            for x in a:
                list_pcd4.append(x)

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.96 µs


In [137]:
len(list_pcd4)

235200

Всего получили 235 000 ключевых фраз. Немало. Перекинем их в датафрейм через служебный лист csv.
В следующей редакции скрипта сделаю это через словарь.

In [138]:
import pandas as pd
df_1 = pd.DataFrame(data={"col1": list_pcd4})
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 235200 entries, 0 to 235199
Data columns (total 1 columns):
col1    235200 non-null object
dtypes: object(1)
memory usage: 1.8+ MB


In [139]:
df_1.to_csv("list_pcd5.csv", sep=',',index=False, encoding='utf-8')

In [140]:
df = pd.read_csv('list_pcd5.csv',sep=',',delimiter=';').reset_index().fillna('')
df.columns = ['pcd','shirina','diametr']
df.head()

,pcd,shirina,diametr
0,3x98,,
1,3x98,4.5,
2,3x98,,13
3,3x98,4.5,13
4,,4.5,13


# Удалим повторяющиеся комбинации параметров

In [141]:
df['concat'] = df['pcd'].astype('str')+df['shirina'].astype('str')+'%'+df['diametr'].astype('str')
df = df.drop_duplicates(subset='concat', keep="first")

In [142]:
df = df.drop(['concat'],axis = 1)

# Переведём числовые колонки в строковый тип, чтобы не получать десятичные дроби в ключевых фразах и параметрах

In [143]:
for i in df.columns:
    df[i] = df[i].astype('str')

In [144]:
df['diametr'] = [x[:-2] for x in df['diametr'].values] #уберём десятичную часть у праметров диаметра

In [145]:
ls = [] #уберём десятичную часть у параметра ширины, если она целая
for x in df['shirina'].values:
    try:
        if x[-1]=='0':
            ls.append(x[:-2])
        else:
            ls.append(x)
    except IndexError:
        ls.append(x)
df['shirina'] = ls
    

# Сгенерируем поисковые запросы
Если в колонке с диаметром пусто,то не будем его выводить

In [146]:
df['zapros'] = '(Диски '+df['pcd']+' '+df['shirina']+\
' '+df['diametr']+'|Диски '+df['pcd']+' '+df['shirina']+\
' R'+df['diametr']+')'


In [147]:
df.loc[df['diametr']=='','zapros'] =  'Диски '+df['pcd']+' '+df['shirina']+' '+\
                                   df['diametr']

In [148]:
df['sluzh'] = 'Диски '+df['pcd'].astype('str')+' '+df['shirina'].astype('str')+' R'+\
                                   df['diametr'].astype('str')
qwerty = df['sluzh'].tolist()
qwerty  = [len(x) for x in qwerty]
print(max(qwerty))

22


### Максимальная длина заголовка - 22 символа, проверяем чтобы работать с шаблонами

In [149]:
df = df.drop(['sluzh'],axis = 1) #удалили служебную колонку

In [150]:
len('Собственный склад. Более 10 000 колёсных дисков в наличии. На любой кошелёк') #измерили длину текстового описания

75

# Сгенерируем все поля для объявлений

In [151]:
%%time
df['title1'] = '#Диски# в наличии в Туле'
df['title2'] = 'Шинный центр Автошина'
df['description'] = 'Собственный склад. Более 10 000 колёсных дисков в наличии. На любой кошелёк.'
df['bs_title'] = 'Акции||Рассрочка||Контакты||Доставка '
df['bs_description'] = 'Актуальные спецпредложения||Процентная ставка 0%. Переплата 0%||Ждём Вас на ул. Рязанской ежедневно||Доставка курьером или самовывоз'
df['bs_links'] = 'http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}||http://xn--e1afdpacvp7ap0b.xn--p1ai/rassrochka?utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}||http://xn--e1afdpacvp7ap0b.xn--p1ai/contacts?utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}||http://xn--e1afdpacvp7ap0b.xn--p1ai/help/delivery?utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}'
df['addictions'] = 'Поиск по авто||Поиск по бренду||Поиск по размеру'

ls = []
for x in df['pcd'].values:
    try:
        ls.append(x[0])
    except IndexError:
        ls.append(x)
df['otverstiya'] = ls


ls = []
for x in df['pcd'].values:
    try:
        ls.append(x[2:])
    except IndexError:
        ls.append(x)
df['d_otvertiy'] = ls

df['link'] = 'http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=param{param1}&utm_source=yandex&utm_medium=cpc&utm_campaign=diski_po_razmery_poisk&utm_term={keyword}'
df['param1'] = '&f%5Bwidth%5D='+df['shirina']+'&f%5Bdiameter%5D='+df['diametr']+'&f%5Bet%5D=&f%5Bpcd_first%5D='+df['otverstiya']+'&f%5Bpcd_second%5D='+df['d_otvertiy']
df['price'] = 3.0
df['region'] = 'Тула'
#df['id'] = [1+x for x in range(df.shape[0])]
#df['id2'] = [1 if '|' in x else 0 for x in df['zapros'].values]
#df['id3'] = [sum(df['id2'].values[:(x+1)]) for x in range(len(df['id2'].values))]
#df['id4'] = df['id']+df['id3']
df['group_ids'] = [(x+1)//100 for x in range(df.shape[0])]
df=df.drop(['pcd','shirina','diametr','otverstiya','d_otvertiy'], axis = 1)


CPU times: user 73 ms, sys: 9.72 ms, total: 82.8 ms
Wall time: 81.4 ms


In [152]:
df.shape[0]

39764

In [153]:

df #посмотрим на данные

,zapros,title1,title2,description,bs_title,bs_description,bs_links,addictions,link,param1,price,region,group_ids
0,Диски 3x98,#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=&f%5Bdiameter%5D=&f%5Bet%5D=&f%5...,3.0,Тула,0
1,Диски 3x98 4.5,#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=4.5&f%5Bdiameter%5D=&f%5Bet%5D=&...,3.0,Тула,0
2,(Диски 3x98 13|Диски 3x98 R13),#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=&f%5Bdiameter%5D=13&f%5Bet%5D=&f...,3.0,Тула,0
3,(Диски 3x98 4.5 13|Диски 3x98 4.5 R13),#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=4.5&f%5Bdiameter%5D=13&f%5Bet%5D...,3.0,Тула,0
4,(Диски 4.5 13|Диски 4.5 R13),#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=4.5&f%5Bdiameter%5D=13&f%5Bet%5D...,3.0,Тула,0
5,Диски 4.5,#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=4.5&f%5Bdiameter%5D=&f%5Bet%5D=&...,3.0,Тула,0
6,(Диски 13|Диски R13),#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=&f%5Bdiameter%5D=13&f%5Bet%5D=&f...,3.0,Тула,0
7,Диски 3х98,#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=&f%5Bdiameter%5D=&f%5Bet%5D=&f%5...,3.0,Тула,0
8,Диски 3х98 4.5,#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Рассрочка||Контакты||Доставка,Актуальные спецпредложения||Процентная ставка ...,http://xn--e1afdpacvp7ap0b.xn--p1ai/actions&ut...,Поиск по авто||Поиск по бренду||Поиск по размеру,http://xn--e1afdpacvp7ap0b.xn--p1ai/discs?tab=...,&f%5Bwidth%5D=4.5&f%5Bdiameter%5D=&f%5Bet%5D=&...,3.0,Тула,0
9,(Диски 3х98 13|Диски 3х98 R13),#Диски# в наличии в Туле,Шинный центр Автошина,Собственный склад. Более 10 000 колёсных диско...,Акции||Расс

In [154]:
qw = []
for i in range(0,380):
    qw.append(df[df['group_ids']==i].shape[0])
max(qw)

100

## Выгрузим экспортный файл в эксель

In [155]:
df.to_csv("export_work.csv", sep=',',index=False, encoding='utf-8')